# Using multi-agent RL with Ray RLlib

Here, we are going to implement a multi-agent RL based on Proximal Policy Optimization (PPO) algorithm using Ray RLlib. We are going to utilize the custom environment RobotsMeeting created in [lesson 3 nb 1](./1-exploring_multi_agent_environment.ipynb).

In [26]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import gymnasium.spaces as spaces
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.tune.registry import register_env
import numpy as np
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm
from typing import Dict
from ray.rllib.policy.policy import PolicySpec

os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output

The cell below implements the environment for the cooperative navigation problem explained in the previous notebook [lesson 3 nb 1](./1-exploring_multi_agent_environment.ipynb).

In [5]:
class RobotsMeeting(
    MultiAgentEnv
):  # We have to inherit from MultiAgentEnv from Ray RLlib similarly to Gymnasium API
    def __init__(
        self,
        scenario_size: int = 10,
        render: bool = False,
    ):
        self.scenario_size = scenario_size  # scenario_size x scenario_size grid
        self.scenario = np.zeros((self.scenario_size, self.scenario_size))
        self.action_space = spaces.Dict(
            {
                "robot_1": spaces.Discrete(4),  # 0: up, 1: down, 2: left, 3: right
                "robot_2": spaces.Discrete(4),
            }
        )
        self.observation_space = spaces.Dict(
            {
                "robot_1": spaces.Discrete(
                    self.scenario_size * 2 - 2
                ),  # The maximum distance between two points in a grid is 2 * scenario_size - 2
                "robot_2": spaces.Discrete(self.scenario_size * 2 - 2),
            }
        )
        self.robots = {
            "robot_1": {
                "number": 1,
                "pos": [0, 0],
            },
            "robot_2": {
                "number": 2,
                "pos": [self.scenario_size - 1, self.scenario_size - 1],
            },
        }
        self.scenario[0, 0] = self.robots["robot_1"]["number"]
        self.scenario[self.scenario_size - 1, self.scenario_size - 1] = self.robots[
            "robot_2"
        ]["number"]
        if render:
            plt.figure()
            plt.show()
            self.render()

    def reset(self):
        # Reseting scenario and returning robots to initial positions
        self.scenario = np.zeros((self.scenario_size, self.scenario_size))
        self.scenario[0, 0] = self.robots["robot_1"]["number"]
        self.scenario[self.scenario_size - 1, self.scenario_size - 1] = self.robots[
            "robot_2"
        ]["number"]
        obs = {
            "robot_1": self.scenario_size * 2 - 2,
            "robot_2": self.scenario_size * 2 - 2,
        }
        info = {}
        return (obs, info)

    def step(self, action_dict):
        for agent, action in action_dict.items():
            if action == 0:  # Up
                if self.robots[agent]["pos"][0] == 0:
                    continue
                self.scenario[
                    self.robots[agent]["pos"][0], self.robots[agent]["pos"][1]
                ] = 0
                self.scenario[
                    self.robots[agent]["pos"][0] - 1, self.robots[agent]["pos"][1]
                ] = self.robots[agent]["number"]
                self.robots[agent]["pos"][0] -= 1
            elif action == 1:  # Down
                if self.robots[agent]["pos"][0] == self.scenario_size - 1:
                    continue
                self.scenario[
                    self.robots[agent]["pos"][0], self.robots[agent]["pos"][1]
                ] = 0
                self.scenario[
                    self.robots[agent]["pos"][0] + 1, self.robots[agent]["pos"][1]
                ] = self.robots[agent]["number"]
                self.robots[agent]["pos"][0] += 1
            elif action == 2:  # Left
                if self.robots[agent]["pos"][1] == 0:
                    continue
                self.scenario[
                    self.robots[agent]["pos"][0], self.robots[agent]["pos"][1]
                ] = 0
                self.scenario[
                    self.robots[agent]["pos"][0], self.robots[agent]["pos"][1] - 1
                ] = self.robots[agent]["number"]
                self.robots[agent]["pos"][1] -= 1
            elif action == 3:  # Right
                if self.robots[agent]["pos"][1] == self.scenario_size - 1:
                    continue
                self.scenario[
                    self.robots[agent]["pos"][0], self.robots[agent]["pos"][1]
                ] = 0
                self.scenario[
                    self.robots[agent]["pos"][0], self.robots[agent]["pos"][1] + 1
                ] = self.robots[agent]["number"]
                self.robots[agent]["pos"][1] += 1
            else:
                raise ValueError("Invalid action")

        distance = np.abs(  # Calculate the distance between two robots
            self.robots["robot_1"]["pos"][0] - self.robots["robot_2"]["pos"][0]
        ) + np.abs(self.robots["robot_1"]["pos"][1] - self.robots["robot_2"]["pos"][1])
        obs = {
            "robot_1": distance,
            "robot_2": distance,
        }
        reward_value = (
            0 if np.isclose(distance, 0) else -1
        )  # Reward is 0 if the robots meet, -1 otherwise
        done = (
            True if reward_value == 0 else False
        )  # Terminate the episode if the robots meet
        reward = {"robot_1": reward_value, "robot_2": reward_value}
        terminated = {"player_1": done, "player_2": done}
        truncated = {"player_1": done, "player_2": done}
        terminated["__all__"], truncated["__all__"] = done, done
        info = {}
        return (obs, reward, terminated, truncated, info)

    def render(self):
        clear_output(wait=True)
        plt.imshow(self.scenario)
        plt.show()

    def close(self):
        plt.close()

Registering our custom environment in the Ray RLlib:

In [13]:
def env_creator(env_config):
    env = RobotsMeeting(scenario_size=10)
    return env


register_env("robots_meeting", lambda config: env_creator(config))

## Creating the multi-agent RL using PPO

In [21]:
def policy_mapping_fn_shared(agent_id, episode=None, worker=None, **kwargs):
    agent_idx = int(agent_id.partition("_")[2])

    return "robot_1" if agent_idx == 1 else "robot_2"

In [22]:
def generate_policies() -> Dict[str, PolicySpec]:
    policies = {
        "robot_1": PolicySpec(),
        "robot_2": PolicySpec(),
    }

    return policies

In [25]:
algo_config = (
    PPOConfig()
    .environment(
        env="robots_meeting",
    )
    .multi_agent(
        policies=generate_policies(),
        policy_mapping_fn=policy_mapping_fn_shared,
        count_steps_by="env_steps",
    )
    .framework("torch")
)

In [27]:
stop = {
    "training_iterations": 100,
}
checkpoint_frequency = 0
store_results_path = str(Path("./ray_results/").resolve()) + "/nb_3/"
agent_name = "ray_ppo"

tuner = tune.Tuner(
    "PPO",
    param_space=algo_config.to_dict(),
    run_config=air.RunConfig(
        storage_path=store_results_path,
        name=agent_name,
        stop=stop,
        verbose=2,
        checkpoint_config=air.CheckpointConfig(
            checkpoint_frequency=checkpoint_frequency,
            checkpoint_at_end=True,
        ),
    ),
)
results = tuner.fit()
print(results)

2024-12-06 02:25:20,541	INFO worker.py:1783 -- Started a local Ray instance.
2024-12-06 02:25:21,105	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-12-06 02:25:21,107	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, act

2024-12-06 02:25:29,978	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_robots_meeting_57723_00000
Traceback (most recent call last):
  File "/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/ray/_private/worker.py", line 2661, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/home/lasse/.local/share/virtualenvs

Trial name
PPO_robots_meeting_57723_00000


2024-12-06 02:25:29,986	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/lasse/ray_minicourse/lesson_3/ray_results/nb_3/ray_ppo' in 0.0033s.
(PPO pid=623065) Install gputil for GPU system monitoring.
(PPO pid=623065) 2024-12-06 02:25:29,974	ERROR actor_manager.py:523 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=623137, ip=200.239.93.233, actor_id=f095575c148e45426b52388301000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fd79a5c2d40>)
(PPO pid=623065)   File "/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/ray/rllib/utils/actor_manager.py", line 192, in apply
(PPO pid=623065)     raise e
(PPO pid=623065)   File "/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/ray/rllib/utils/actor_manager.py", line 181, in apply
(PPO pid=623065)     return func(self, *args, **kwargs)
(PPO pid=623065)   File "/home/

ResultGrid<[
  Result(
    error='RayTaskError(TypeError)',
    metrics={},
    path='/home/lasse/ray_minicourse/lesson_3/ray_results/nb_3/ray_ppo/PPO_robots_meeting_57723_00000_0_2024-12-06_02-25-21',
    filesystem='local',
    checkpoint=None
  )
]>
